In [1]:
import pandas as pd
from math import comb
from ax.service.ax_client import AxClient
import time
import sys

sys.path.append('../../py_files')
from sdlnano import BayesianOptimizer_Auto
from calculation import BatchCalculation

In [ ]:
# Design space size calculation


def calculate_combinations(num_variables, total_sum, step):
    steps = int(total_sum / step)
    return comb(steps + num_variables - 1, num_variables - 1)

# Define parameters
step = 0.05
total_sum = 1

# Calculate combinations for each group
count_group1 = calculate_combinations(10, total_sum, step)  # 3 Solid lipids + 3 Polymers + 3 Liquid lipids + 1 Drug
count_group2 = calculate_combinations(4, total_sum, step)   # 3 Surfactants + 1 Water

# Calculate total combinations
total_combinations = count_group1 * count_group2


print("organic design space size: ", round(count_group1/(1e6),2), 'Million')
print("aqueous design space size: ", round(count_group2/(1e3),2), 'Thousand')
print("design space size: ", round(total_combinations/(1e9),2), 'Billion')


organic design space size:  10.02 Million
aqueous design space size:  1.77 Thousand
design space size:  17.74 Billion


# Check the iteration number

In [3]:
iteration = BayesianOptimizer_Auto.get_iteration_number()
print("This is the iteration: #", iteration)

This is the iteration: # 8


# Check the drug

In [4]:
drug = 'ACE'
print("This is the drug:", drug)

This is the drug: ACE


# Optimizer initialization (N/A)

# Generate recommendations

In [5]:
time_start = time.time()

previous_optimizer_file_name = '../iteration_' + str(iteration-1) + '/optimizer/optimizer_load_' + str(iteration-1) + '.json'
# generate recomms
ax_client_init = AxClient.load_from_json_file(previous_optimizer_file_name)
unlabeled_trial, ax_client_design = BayesianOptimizer_Auto.generate_trials(ax_client=ax_client_init, num_of_trials = 16, drug = drug, bopt=1)

time_end = time.time()

print(round((time_end-time_start)/60), 'min')

[INFO 02-12 12:49:06] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[INFO 02-12 12:50:01] ax.service.ax_client: Generated new trial 128 with parameters {'Drug_MW': 0.354, 'Drug_LogP': 0.391, 'Drug_TPSA': 0.067, 'Drug': 0.990368, 'SL_1': 0.0, 'SL_2': 0.0, 'SL_3': 0.0, 'LL_1': 0.0, 'LL_2': 0.213464, 'LL_3': 0.0, 'P_1': 0.0, 'P_2': 0.086214, 'P_3': 0.899215, 'S_1': 0.0, 'S_2': 0.816251, 'S_3': 0.0, 'Water': 0.389424} using model BoTorch.
/Users/zeqing/opt/anaconda3/envs/sdlnano/lib/python3.10/site-packages/ax/core/data.py:288: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return cls(df=pd.co

13 min


In [ ]:
# process the trails into ratios

unlabeled_trial_processed = BayesianOptimizer_Auto.process_trails(unlabeled_trial)
unlabeled_trial_processed.describe()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.0,16.00,16.0,16.000000,16.0,16.000000,16.000000,16.000000,16.000000,16.0,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000
mean,135.500000,354.0,3.91,67.0,0.343875,0.0,0.017812,0.015000,0.044875,0.148250,0.0,0.059250,0.073063,0.297687,0.012438,0.496687,0.201437,0.289563
std,4.760952,0.0,0.00,0.0,0.105729,0.0,0.071250,0.055629,0.096596,0.095046,0.0,0.089458,0.084023,0.124096,0.049750,0.217275,0.247531,0.246177
min,128.000000,354.0,3.91,67.0,0.223000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.146000,0.000000,0.207000,0.000000,0.000000
25%,131.750000,354.0,3.91,67.0,0.251500,0.0,0.000000,0.000000,0.000000,0.079000,0.0,0.000000,0.000000,0.191500,0.000000,0.374250,0.000000,0.000000
50%,135.500000,354.0,3.91,67.0,0.317000,0.0,0.000000,0.000000,0.000000,0.117000,0.0,0.002500,0.021500,0.285500,0.000000,0.437500,0.076000,0.360000
75%,139.250000,354.0,3.91,67.0,0.453750,0.0,0.000000,0.000000,0.000000,0.246750,0.0,0.144000,0.164000,0.417750,0.000000,0.538500,0.412500,0.505250
max,143.000000,354.0,3.91,67.0,0.505000,0.0,0.285000,0.223000,0.253000,0.313000,0.0,0.230000,0.198000,0.490000,0.199000,1.000000,0.647000,0.607000


In [7]:
unlabeled_trial
unlabeled_trial.describe()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000,16.000,16.000,16.000000,16.0,16.000000,16.000000,16.000000,16.000000,16.0,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000
mean,135.500000,0.354,0.391,0.067,0.978126,0.0,0.053525,0.040485,0.187500,0.509320,0.0,0.216448,0.279749,0.827045,0.029255,0.725523,0.331871,0.554159
std,4.760952,0.000,0.000,0.000,0.027377,0.0,0.214100,0.152865,0.403113,0.384281,0.0,0.326432,0.342891,0.157404,0.117020,0.209511,0.440967,0.480681
min,128.000000,0.354,0.391,0.067,0.924643,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.495052,0.000000,0.226689,0.000000,0.000000
25%,131.750000,0.354,0.391,0.067,0.958964,0.0,0.000000,0.000000,0.000000,0.201818,0.0,0.000000,0.000000,0.791230,0.000000,0.622208,0.000000,0.000000
50%,135.500000,0.354,0.391,0.067,0.995184,0.0,0.000000,0.000000,0.000000,0.342182,0.0,0.005120,0.048960,0.887146,0.000000,0.757504,0.099834,0.738557
75%,139.250000,0.354,0.391,0.067,1.000000,0.0,0.000000,0.000000,0.000000,0.950114,0.0,0.518973,0.608639,0.918463,0.000000,0.887256,0.848331,1.000000
max,143.000000,0.354,0.391,0.067,1.000000,0.0,0.856399,0.612794,1.000000,1.000000,0.0,0.788134,0.872247,1.000000,0.468081,1.000000,1.000000,1.000000


In [ ]:
# Convert ratios to volumes

transfer = BatchCalculation.converter(unlabeled_trial_processed)
transfer.describe()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Solvent
count,16.000000,16.0,16.00,16.0,16.000000,16.0,16.000000,16.000000,16.000000,16.000000,16.0,16.000000,16.000000,16.000000,16.0000,16.000000,16.00000,16.000000,16.000000
mean,135.500000,354.0,3.91,67.0,120.356250,0.0,6.234375,5.250000,15.706250,51.887500,0.0,20.737500,25.571875,104.190625,12.4375,496.687500,201.43750,289.562500,0.065625
std,4.760952,0.0,0.00,0.0,37.004977,0.0,24.937500,19.469985,33.808593,33.265961,0.0,31.310299,29.408185,43.433429,49.7500,217.274855,247.53073,246.177434,0.262500
min,128.000000,354.0,3.91,67.0,78.050000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,51.100000,0.0000,207.000000,0.00000,0.000000,-0.350000
25%,131.750000,354.0,3.91,67.0,88.025000,0.0,0.000000,0.000000,0.000000,27.650000,0.0,0.000000,0.000000,67.025000,0.0000,374.250000,0.00000,0.000000,0.000000
50%,135.500000,354.0,3.91,67.0,110.950000,0.0,0.000000,0.000000,0.000000,40.950000,0.0,0.875000,7.525000,99.925000,0.0000,437.500000,76.00000,360.000000,0.000000
75%,139.250000,354.0,3.91,67.0,158.812500,0.0,0.000000,0.000000,0.000000,86.362500,0.0,50.400000,57.400000,146.212500,0.0000,538.500000,412.50000,505.250000,0.350000
max,143.000000,354.0,3.91,67.0,176.750000,0.0,99.750000,78.050000,88.550000,109.550000,0.0,80.500000,69.300000,171.500000,199.0000,1000.000000,647.00000,607.000000,0.350000


In [9]:
#dead_volume = 3.0 # mL

round(transfer[['Drug','SL_1', 'SL_2', 'SL_3', 'LL_1', 'LL_2', 'LL_3', 'P_1', 'P_2', 'P_3', 'S_1', 'S_2', 'S_3', 'Water', 'Solvent']].sum()/1000,1)

Drug       1.9
SL_1       0.0
SL_2       0.1
SL_3       0.1
LL_1       0.3
LL_2       0.8
LL_3       0.0
P_1        0.3
P_2        0.4
P_3        1.7
S_1        0.2
S_2        7.9
S_3        3.2
Water      4.6
Solvent    0.0
dtype: float64

In [10]:
# file path
ax_client_design_path = "optimizer/optimizer_design_" + str(iteration) + ".json"

unlabeled_trial_path = "data/unlabeled_" + str(iteration) + ".xlsx"

unlabeled_trial_processed_path = "data/unlabeled_processed_" + str(iteration) + ".xlsx"

transfer_path = 'data/transfer_' + str(iteration) + '.xlsx'


In [11]:
ax_client_design.save_to_json_file(ax_client_design_path)

unlabeled_trial.to_excel(unlabeled_trial_path, index=False)

unlabeled_trial_processed.to_excel(unlabeled_trial_processed_path, index=False)

transfer.drop(['Drug_MW', 'Drug_LogP', 'Drug_TPSA'], axis=1).to_excel(transfer_path, index = False)

[INFO 02-12 13:02:55] ax.service.ax_client: Saved JSON-serialized state of optimization to `optimizer/optimizer_design_8.json`.


# Generate new protocol

In [12]:
# Paths
template_protocol = '../../templates/OT_protocol_template.py'

output_script_path = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_.py'


In [13]:
# Update the script with Excel data
updated_script = BatchCalculation.update_transfer_script(template_protocol, transfer_path, output_script_path) 


In [14]:
updated_script

'protocol/iteration_8_OT_2_protocol_.py'

# Perform experiment

# Results

In [15]:
results = BatchCalculation.parameters_and_complexity(iteration)
results

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Complexity,Complexity_STD
0,128,354,3.91,67,0.452,0,0.000,0.000,0.000,0.098,0,0.000,0.039,0.411,0.000,0.677,0.000,0.323,4,0
1,129,354,3.91,67,0.333,0,0.285,0.000,0.000,0.076,0,0.000,0.004,0.303,0.000,0.442,0.000,0.558,5,0
2,130,354,3.91,67,0.486,0,0.000,0.000,0.000,0.051,0,0.005,0.000,0.458,0.000,0.372,0.078,0.550,5,0
3,131,354,3.91,67,0.505,0,0.000,0.000,0.000,0.000,0,0.005,0.000,0.490,0.000,0.393,0.000,0.607,3,0
4,132,354,3.91,67,0.411,0,0.000,0.000,0.000,0.136,0,0.000,0.084,0.368,0.199,0.375,0.000,0.426,5,0
5,133,354,3.91,67,0.364,0,0.000,0.223,0.000,0.086,0,0.000,0.000,0.326,0.000,0.433,0.089,0.479,5,0
6,134,354,3.91,67,0.227,0,0.000,0.000,0.227,0.186,0,0.015,0.198,0.146,0.000,0.654,0.346,0.000,7,0
7,135,354,3.91,67,0.238,0,0.000,0.000,0.238,0.145,0,0.000,0.179,0.200,0.000,0.926,0.074,0.000,6,0
8,136,354,3.91,67,0.267,0,0.000,0.000,0.000,0.265,0,0.139,0.173,0.155,0.000,0.447,0.553,0.000,6,0
9,137,354,3.91,67,0.253,0,0.000,0.000,0.253,0.091,0,0.000,0.186,0.218,0.000,0.479,0.000,0.521,5,0


In [16]:
size_raw = BatchCalculation.size_raw(iteration)
size_raw

,Data Quality,Item,Size,PD Index
0,Good,A1,148.104320,0.315835
1,Good,A2,157.017385,0.237230
2,Good,A3,151.349021,0.276512
3,Good,A4,266.916394,0.362057
4,Good,A5,255.352863,0.408333
5,Good,A6,246.568261,0.460926
6,Good,B1,73.941437,0.499604
7,Good,B2,72.755213,0.440204
8,Good,B3,81.452286,0.416267
9,Good,B4,32.026646,0.198020


In [17]:
size_processed = BatchCalculation.process_formulations(size_raw)
size_processed

,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,Formulation 1,152.156909,4.511119,0.276526,0.039303,1
1,Formulation 2,256.279173,10.205644,0.410439,0.049468,1
2,Formulation 3,76.049645,4.716267,0.452025,0.042908,1
3,Formulation 4,31.299594,1.028207,0.223943,0.036661,1
4,Formulation 5,199.664637,8.519157,0.387497,0.027139,1
5,Formulation 6,270.571357,2.881870,0.336123,0.045773,1
6,Formulation 7,194.946835,7.242944,0.203145,0.025373,1
7,Formulation 8,174.063132,2.621943,0.216969,0.027155,1
8,Formulation 9,219.172087,3.601228,0.197554,0.036311,1
9,Formulation 10,127.370244,1.920557,0.281679,0.033511,1


In [18]:
results_final = pd.concat([results, size_processed], axis=1)
results_final

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,...,S_3,Water,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,128,354,3.91,67,0.452,0,0.000,0.000,0.000,0.098,...,0.000,0.323,4,0,Formulation 1,152.156909,4.511119,0.276526,0.039303,1
1,129,354,3.91,67,0.333,0,0.285,0.000,0.000,0.076,...,0.000,0.558,5,0,Formulation 2,256.279173,10.205644,0.410439,0.049468,1
2,130,354,3.91,67,0.486,0,0.000,0.000,0.000,0.051,...,0.078,0.550,5,0,Formulation 3,76.049645,4.716267,0.452025,0.042908,1
3,131,354,3.91,67,0.505,0,0.000,0.000,0.000,0.000,...,0.000,0.607,3,0,Formulation 4,31.299594,1.028207,0.223943,0.036661,1
4,132,354,3.91,67,0.411,0,0.000,0.000,0.000,0.136,...,0.000,0.426,5,0,Formulation 5,199.664637,8.519157,0.387497,0.027139,1
5,133,354,3.91,67,0.364,0,0.000,0.223,0.000,0.086,...,0.089,0.479,5,0,Formulation 6,270.571357,2.881870,0.336123,0.045773,1
6,134,354,3.91,67,0.227,0,0.000,0.000,0.227,0.186,...,0.346,0.000,7,0,Formulation 7,194.946835,7.242944,0.203145,0.025373,1
7,135,354,3.91,67,0.238,0,0.000,0.000,0.238,0.145,...,0.074,0.000,6,0,Formulation 8,174.063132,2.621943,0.216969,0.027155,1
8,136,354,3.91,67,0.267,0,0.000,0.000,0.000,0.265,...,0.553,0.000,6,0,Formulation 9,219.172087,3.601228,0.197554,0.036311,1
9,137,354,3.91,67,0.253,0,0.000,0.000,0.253,0.091,...,0.000,0.521,5,0,Formulation 10,127.370244,1.920557,0.281679,0.033511,1


In [19]:
# µg/mL
results_final['Solu'] = results_final['Drug'] * 600 / 0.3
results_final['Solu_STD'] = 0

results_final.loc[results_final['Formulation Quality'] == 0, 'Solu'] = 0
results_final

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,128,354,3.91,67,0.452,0,0.000,0.000,0.000,0.098,...,4,0,Formulation 1,152.156909,4.511119,0.276526,0.039303,1,904.0,0
1,129,354,3.91,67,0.333,0,0.285,0.000,0.000,0.076,...,5,0,Formulation 2,256.279173,10.205644,0.410439,0.049468,1,666.0,0
2,130,354,3.91,67,0.486,0,0.000,0.000,0.000,0.051,...,5,0,Formulation 3,76.049645,4.716267,0.452025,0.042908,1,972.0,0
3,131,354,3.91,67,0.505,0,0.000,0.000,0.000,0.000,...,3,0,Formulation 4,31.299594,1.028207,0.223943,0.036661,1,1010.0,0
4,132,354,3.91,67,0.411,0,0.000,0.000,0.000,0.136,...,5,0,Formulation 5,199.664637,8.519157,0.387497,0.027139,1,822.0,0
5,133,354,3.91,67,0.364,0,0.000,0.223,0.000,0.086,...,5,0,Formulation 6,270.571357,2.881870,0.336123,0.045773,1,728.0,0
6,134,354,3.91,67,0.227,0,0.000,0.000,0.227,0.186,...,7,0,Formulation 7,194.946835,7.242944,0.203145,0.025373,1,454.0,0
7,135,354,3.91,67,0.238,0,0.000,0.000,0.238,0.145,...,6,0,Formulation 8,174.063132,2.621943,0.216969,0.027155,1,476.0,0
8,136,354,3.91,67,0.267,0,0.000,0.000,0.000,0.265,...,6,0,Formulation 9,219.172087,3.601228,0.197554,0.036311,1,534.0,0
9,137,354,3.91,67,0.253,0,0.000,0.000,0.253,0.091,...,5,0,Formulation 10,127.370244,1.920557,0.281679,0.033511,1,506.0,0


In [20]:
results_normalized = BayesianOptimizer_Auto.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,1,0.152157,0.004511,0.452,0.0,0.276526,0.039303,0.333333
1,1,0.256279,0.010206,0.333,0.0,0.410439,0.049468,0.416667
2,1,0.076050,0.004716,0.486,0.0,0.452025,0.042908,0.416667
3,1,0.031300,0.001028,0.505,0.0,0.223943,0.036661,0.250000
4,1,0.199665,0.008519,0.411,0.0,0.387497,0.027139,0.416667
5,1,0.270571,0.002882,0.364,0.0,0.336123,0.045773,0.416667
6,1,0.194947,0.007243,0.227,0.0,0.203145,0.025373,0.583333
7,1,0.174063,0.002622,0.238,0.0,0.216969,0.027155,0.500000
8,1,0.219172,0.003601,0.267,0.0,0.197554,0.036311,0.500000
9,1,0.127370,0.001921,0.253,0.0,0.281679,0.033511,0.416667


In [21]:
results_normalized = BayesianOptimizer_Auto.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,1,0.152157,0.004511,0.452,0.0,0.276526,0.039303,0.333333
1,1,0.256279,0.010206,0.333,0.0,0.410439,0.049468,0.416667
2,1,0.076050,0.004716,0.486,0.0,0.452025,0.042908,0.416667
3,1,0.031300,0.001028,0.505,0.0,0.223943,0.036661,0.250000
4,1,0.199665,0.008519,0.411,0.0,0.387497,0.027139,0.416667
5,1,0.270571,0.002882,0.364,0.0,0.336123,0.045773,0.416667
6,1,0.194947,0.007243,0.227,0.0,0.203145,0.025373,0.583333
7,1,0.174063,0.002622,0.238,0.0,0.216969,0.027155,0.500000
8,1,0.219172,0.003601,0.267,0.0,0.197554,0.036311,0.500000
9,1,0.127370,0.001921,0.253,0.0,0.281679,0.033511,0.416667


In [22]:
labeled_data_path = 'data/labeled_' + str(iteration) + '.xlsx'
labeled_norm_path = 'data/labeled_norm_' + str(iteration) + '.xlsx'


In [23]:
results_final.to_excel(labeled_data_path, index=False)
results_normalized.to_excel(labeled_norm_path, index=False)

# Load the labeled data to update the optimizer

In [24]:
labeled_norm = pd.read_excel(labeled_norm_path)
labeled_norm.head()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,128,354,3.91,67,0.452,0,0.000,0.0,0.0,0.098,...,0.333333,0,Formulation 1,0.152157,0.004511,0.276526,0.039303,1,0.452,0
1,129,354,3.91,67,0.333,0,0.285,0.0,0.0,0.076,...,0.416667,0,Formulation 2,0.256279,0.010206,0.410439,0.049468,1,0.333,0
2,130,354,3.91,67,0.486,0,0.000,0.0,0.0,0.051,...,0.416667,0,Formulation 3,0.076050,0.004716,0.452025,0.042908,1,0.486,0
3,131,354,3.91,67,0.505,0,0.000,0.0,0.0,0.000,...,0.250000,0,Formulation 4,0.031300,0.001028,0.223943,0.036661,1,0.505,0
4,132,354,3.91,67,0.411,0,0.000,0.0,0.0,0.136,...,0.416667,0,Formulation 5,0.199665,0.008519,0.387497,0.027139,1,0.411,0


In [25]:
ax_client_design = AxClient.load_from_json_file(ax_client_design_path)


[INFO 02-12 15:27:05] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.


In [26]:
loaded_optimizer_path = "optimizer/optimizer_load_" + str(iteration) + ".json"

In [27]:
loaded_ax_client = BayesianOptimizer_Auto.load_labeled_data(ax_client_design, labeled_norm_path)
loaded_ax_client.save_to_json_file(loaded_optimizer_path)

[INFO 02-12 15:27:05] ax.service.ax_client: Completed trial 128 with data: {'Solu': (0.452, 0), 'Size': (0.152157, 0.004511), 'PDI': (0.276526, 0.039303), 'Complexity': (0.333333, 0)}.
[INFO 02-12 15:27:05] ax.service.ax_client: Completed trial 129 with data: {'Solu': (0.333, 0), 'Size': (0.256279, 0.010206), 'PDI': (0.410439, 0.049468), 'Complexity': (0.416667, 0)}.
[INFO 02-12 15:27:05] ax.service.ax_client: Completed trial 130 with data: {'Solu': (0.486, 0), 'Size': (0.07605, 0.004716), 'PDI': (0.452025, 0.042908), 'Complexity': (0.416667, 0)}.
[INFO 02-12 15:27:05] ax.service.ax_client: Completed trial 131 with data: {'Solu': (0.505, 0), 'Size': (0.0313, 0.001028), 'PDI': (0.223943, 0.036661), 'Complexity': (0.25, 0)}.
[INFO 02-12 15:27:05] ax.service.ax_client: Completed trial 132 with data: {'Solu': (0.411, 0), 'Size': (0.199665, 0.008519), 'PDI': (0.387497, 0.027139), 'Complexity': (0.416667, 0)}.
[INFO 02-12 15:27:05] ax.service.ax_client: Completed trial 133 with data: {'Solu'